In [1]:
# Basic environment modules
import numpy as np
import pandas as pd
import psycopg2 as pg # PostgreSQL module
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import font_manager
import matplotlib.font_manager as fm

# Data analysis modules
from CoolProp.CoolProp import PropsSI
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from scipy.integrate import ode
from scipy.interpolate import griddata
from sqlalchemy import create_engine

# User-defined libraries
from Model import *
from PhysicalProperty import *
from StructuredQuery import *
from Numeric import *
from Authentication import *

# Set the display format to show 6 decimal places
pd.options.display.float_format = '{:.6f}'.format

print("Import modules successfully.")

# load class instances
sql = StructuredQuery()
pro = PhysicalProperty()
mod = Model()
oau = Authentication()

Import modules successfully.
Construct connection string and update
SQL library is successfully started.
Property_OSV is successfully started.
Models of boiling heat transfer is successfully started.
Authentication is successfully started.


In [2]:
# Connect to DB Server and get data
# Load json file (Security)
json_parse = oau.get_apikey(json_filename='key.json')
json_res = pd.json_normalize(json_parse['sql'])

# Set query from DB
sql = "SELECT * FROM CHF.raw_database WHERE geo IN ('C','A')"
#sql ="SELECT * FROM CHF.raw_database WHERE refri IN ('R12', 'R113', 'R114', 'R21') AND idx != '31'"
#sql = "SELECT * FROM CHF.raw_database WHERE refri IN ('H2O', 'D2O') AND idx != '31'"
#sql = "SELECT * FROM CHF.raw_database WHERE idx = '12' AND tno = 'T6'"

# Connect DB server
#(conn, db_engine) = sql.connect(json_res["host"][1], json_res["dbname"][1], json_res["user"][1], json_res["port"][1], json_res["password"][1], json_res["service"][1]) #postgreSQL
connect = pys.connect(host=json_res["host"][1], user = json_res["user"][1], password = json_res["password"][1], cursorclass=pys.cursors.DictCursor)
cur = connect.cursor()
cur.execute(sql)


result = cur.fetchall()
raw_tb = pd.DataFrame(result)

del result, sql

# Set query from DB
sql = "SELECT * FROM CHF.physicalproperties"
cur.execute(sql)
result = cur.fetchall()
pp_tb = pd.DataFrame(result)

cur.close()
del result, sql, connect # delete quiry log

print("Loading database completed. data size: {}".format(len(raw_tb)))
print("Loading database completed. data size: {}".format(len(pp_tb)))

Loading database completed. data size: 20315
Loading database completed. data size: 170


In [3]:
# Calculation of physical properties

# if physical properties can't calculated, the other step is need (interpolation based on the physical properties table) = Sodium
prop_tb = raw_tb.copy()

"""
R113, R114, R21 can't use viscosity, thermal conductivity model -> re-mapping these models based on interpolation tables
R12 Pressure range : 0.242551~4.1361e+06 Pa (P, Q Flash)
H2O Pressure range : 611.655~2.2064e+07 Pa (P, Q Flash)
ValueError: For now, we don't support T [273.545 K] below Tmin(saturation) [276.969 K] : PropsSI("H","T",273.545,"P",6894757.1,"D2O") -> convert to D2O -> H2O
"""

# start the timer
start_time = time.time()
print('START TIME :',str(datetime.now())[10:19] )

# Set osv correlation method
m_idx = 1

# compute physical properties
for i in range(0, len(prop_tb)):
    # calculate default parameter
    prop_tb.loc[i, 'qt'] = prop_tb.loc[i, 'qi'] + prop_tb.loc[i, 'qo']
    # initialize heat flux
    try:
        if prop_tb.loc[i, 'hs'] == 3:
            prop_tb.loc[i, 'qav'] = 1/(0.5*(1/prop_tb.loc[i, 'qi']+1/prop_tb.loc[i, 'qo']))
        else:
            prop_tb.loc[i, 'qav'] = prop_tb.loc[i, 'qt']
    except:
        prop_tb.loc[i, 'qav'] = prop_tb.loc[i, 'qt']

    prop_tb.loc[i, 'mass'] = round(PropsSI(prop_tb.loc[i, 'refri'], 'molemass') * 1e3,6)
    prop_tb.loc[i, 'de'], prop_tb.loc[i, 'ph'], prop_tb.loc[i,'af'] = pro.cal_de(prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'lh'])
    prop_tb.loc[i, 'lpd'] = prop_tb.loc[i, 'lh']/prop_tb.loc[i, 'dhy'] # Lh / dh
    if prop_tb.loc[i, 'geo'] == 'C':
        prop_tb.loc[i, 'qr'] = 1
    else:
        if prop_tb.loc[i, 'hs'] == 1:
            prop_tb.loc[i, 'qr'] = 0
        elif prop_tb.loc[i, 'hs'] == 2:
            prop_tb.loc[i, 'qr'] = 1
        else:
            prop_tb.loc[i, 'qr'] = prop_tb.loc[i, 'qo']/prop_tb.loc[i, 'qi'] 

    # calculate physical properties at saturation point
    prop_tb.loc[i,'tsat'] = round(PropsSI('T', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'pcrit'] = round(PropsSI(prop_tb.loc[i, 'refri'],'pcrit') * 10**-5, 12)
    prop_tb.loc[i,'tcrit'] = round(PropsSI('Tcrit', prop_tb.loc[i, 'refri']), 12)
    prop_tb.loc[i,'rhocrit'] = round(PropsSI('rhocrit', prop_tb.loc[i, 'refri']), 12)
    prop_tb.loc[i,'zc'] = round(PropsSI('Z', prop_tb.loc[i, 'refri']), 12)
    prop_tb.loc[i,'rdcp'] = round(prop_tb.loc[i, 'pre']/prop_tb.loc[i, 'pcrit'],12)
    prop_tb.loc[i,'rhof'] = round(PropsSI('D', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'rhov'] = round(PropsSI('D', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri']),12)
    
    # set coefficient parameter
    if prop_tb.loc[i, 'refri'] == 'R113' or prop_tb.loc[i, 'refri'] == 'R114' or prop_tb.loc[i, 'refri'] == 'R21':
        prop_tb.loc[i,'muf'] = griddata(pp_tb[pp_tb['refri'] == prop_tb.loc[i, 'refri']]['p'], pp_tb['muf'], prop_tb.loc[i, 'pre'], method='linear')
        prop_tb.loc[i,'muv'] = griddata(pp_tb[pp_tb['refri'] == prop_tb.loc[i, 'refri']]['p'], pp_tb['muv'], prop_tb.loc[i, 'pre'], method='linear')
        prop_tb.loc[i, 'kf'] = griddata(pp_tb[pp_tb['refri'] == prop_tb.loc[i, 'refri']]['p'], pp_tb['kf'], prop_tb.loc[i, 'pre'], method='linear')
        prop_tb.loc[i, 'kv'] = griddata(pp_tb[pp_tb['refri'] == prop_tb.loc[i, 'refri']]['p'], pp_tb['kv'], prop_tb.loc[i, 'pre'], method='linear')
    else:
        prop_tb.loc[i,'muf'] = round(PropsSI('V', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12)
        prop_tb.loc[i,'muv'] = round(PropsSI('V', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri']),12)
        prop_tb.loc[i, 'kf'] = round(PropsSI('L', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12) # [W/m/K]
        prop_tb.loc[i, 'kv'] = round(PropsSI('L', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri']),12) # [W/m/K]
    
    prop_tb.loc[i,'hfo'] = round(PropsSI('H', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
    prop_tb.loc[i,'hvo'] = round(PropsSI('H', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
    prop_tb.loc[i, 'lam'] = round((prop_tb.loc[i, 'hvo'] - prop_tb.loc[i,'hfo']),12) # [kJ/kg]

    # calculate physical propoerties of water at same conditions
    prop_tb.loc[i,'wtsat'] = round(PropsSI('T', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12)
    prop_tb.loc[i,'wpcrit'] = round(PropsSI(prop_tb.loc[i, 'refri'],'pcrit') * 10**-5, 12)
    prop_tb.loc[i,'wrdcp'] = round(prop_tb.loc[i, 'pre']/prop_tb.loc[i,'pcrit'],12)
    prop_tb.loc[i,'wrhof'] = round(PropsSI('D', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12)
    prop_tb.loc[i,'wrhov'] = round(PropsSI('D', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O'),12)
    prop_tb.loc[i,'wmuf'] = round(PropsSI('V', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12)
    prop_tb.loc[i,'wmuv'] = round(PropsSI('V', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O'),12)
    prop_tb.loc[i,'whfo'] = round(PropsSI('H', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O')*1e-3,12) #[kJ/kg]
    prop_tb.loc[i,'whvo'] = round(PropsSI('H', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O')*1e-3,12) #[kJ/kg]
    prop_tb.loc[i, 'wlam'] = round((prop_tb.loc[i, 'whvo'] - prop_tb.loc[i,'whfo']),12) # [kJ/kg]

    # calculate flow parameters
    if pd.isna(prop_tb.loc[i, 'vf']):
        prop_tb.loc[i, 'vf'] = round(prop_tb.loc[i, 'gre'] / prop_tb.loc[i, 'rhof'], 12) # [m/s]
    else:
        if pd.isna(prop_tb.loc[i, 'gre']):
            prop_tb.loc[i, 'gre'] = round(prop_tb.loc[i, 'vf']*prop_tb.loc[i, 'rhof'], 12) # [kg/m2s]
    prop_tb.loc[i, 'cpf']   = round(PropsSI('C', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12) # [J/kgK]
    prop_tb.loc[i, 'cpv']   = round(PropsSI('C', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri']),12) # [J/kgK]
    prop_tb.loc[i, 'sigma'] = round(PropsSI('I', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12) # [N/m]
    
    prop_tb.loc[i, 'wcpf']   = round(PropsSI('C', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12) # [J/kgK]
    prop_tb.loc[i, 'wcpv']   = round(PropsSI('C', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O'),12) # [J/kgK]
    prop_tb.loc[i, 'wsigma'] = round(PropsSI('I', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12) # [N/m]
    prop_tb.loc[i, 'wkf'] = round(PropsSI('L', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12) # [W/m/K]
    prop_tb.loc[i, 'wkv'] = round(PropsSI('L', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O'),12) # [W/m/K]
    
    # enthin is empty enthin = hsat - he
    # Fill Na related to inlet subcooling
    if pd.isna(prop_tb.loc[i, 'xi']):
        # Xi is NaN
        if pd.isna(prop_tb.loc[i, 'enthin']):
            # Xi, enthin is NaN
            if pd.isna(prop_tb.loc[i, 'degtin']):
                # Xi, enthin, degtin is NaN
                if pd.isna(prop_tb.loc[i, 'tin']):
                    # Xi, enthin, degtin, and tin is NaN
                    #print("idx: {} and {}th data is not inlet condition".format(prop_tb.loc[i, 'idx'], i))
                    prop_tb.loc[i, 'xi'] = pro.cal_xi(prop_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xe'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 0)
                    prop_tb.loc[i, 'enthin'] = round(-prop_tb.loc[i, 'xi']*prop_tb.loc[i, 'lam'], 12)
                    prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]
                    prop_tb.loc[i, 'degtin'] = round((-prop_tb.loc[i, 'enthin']*1e-3)/prop_tb.loc[i, 'cpf'], 12) # degC
                    prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] -273.15 # degC
                else:
                    # ONLY tin exist
                    prop_tb.loc[i, 'hin'] = round(PropsSI('H', 'T', prop_tb.loc[i, 'tin'] +273.15, 'P', prop_tb.loc[i, 'pre'] * 1e5, prop_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
                    prop_tb.loc[i, 'enthin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'hin'], 6)  # [kJ/kg]
                    prop_tb.loc[i, 'degtin'] = round(prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'tin'], 6) # [degC]
                    prop_tb.loc[i, 'xi'] = round(-prop_tb.loc[i, 'enthin']/prop_tb.loc[i, 'lam'],12)
            else:
                # ONLY degtin exist
                prop_tb.loc[i, 'xi'] = round(-prop_tb.loc[i, 'cpf']*prop_tb.loc[i, 'degtin']*1e-3/prop_tb.loc[i, 'lam'],6) #[-]
                prop_tb.loc[i, 'enthin'] = round(-prop_tb.loc[i, 'cpf'] * prop_tb.loc[i, 'degtin']*1e-3, 6) # [kJ/kg]
                prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]
                prop_tb.loc[i, 'tin'] = round(prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] -273.15, 6) # degC
        else:
            # enthin exist
            prop_tb.loc[i, 'xi'] = round(-prop_tb.loc[i, 'enthin']/prop_tb.loc[i, 'lam'], 6)
            prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]            
            prop_tb.loc[i, 'degtin'] = round(-(prop_tb.loc[i, 'enthin']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
            prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] -273.15 # degC
    else:
        # Xi exist
        prop_tb.loc[i, 'enthin'] = round(-prop_tb.loc[i, 'xi']*prop_tb.loc[i, 'lam'], 12)
        prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]
        prop_tb.loc[i, 'degtin'] = round((-prop_tb.loc[i, 'enthin']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
        prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] -273.15 # degC

    # enthout is empty enthout = he - hsat
    # Fill Na related to outlet subcooling
    if pd.isna(prop_tb.loc[i, 'xe']):
        # Xe is NaN
        if pd.isna(prop_tb.loc[i, 'enthout']):
            # Xe, enthout is NaN
            if pd.isna(prop_tb.loc[i, 'degtout']):
                # Xe, enthout, degtout is NaN
                if pd.isna(prop_tb.loc[i, 'tout']):
                    # Xe, enthout, degtout, and tout is NaN
                    #print("idx: {} and {}th data is not outlet condition".format(prop_tb.loc[i, 'idx'], i))
                    prop_tb.loc[i, 'xe'] = pro.cal_xe(prop_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xi'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 0)
                    prop_tb.loc[i, 'enthout'] = round(prop_tb.loc[i, 'xe']*prop_tb.loc[i, 'lam'], 12)
                    prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthout'], 12) # [kJ/kg]
                    prop_tb.loc[i, 'degtout'] = round((prop_tb.loc[i, 'enthout']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
                    prop_tb.loc[i, 'tout'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtout'] -273.15 # degC
                else:
                    # ONLY tout exist
                    prop_tb.loc[i, 'hout'] = round(PropsSI('H', 'T', prop_tb.loc[i, 'tout'] +273.15, 'P', prop_tb.loc[i, 'pre'] * 1e5, prop_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
                    prop_tb.loc[i, 'enthout'] = prop_tb.loc[i, 'hout'] - prop_tb.loc[i, 'hfo']  # [kJ/kg]
                    prop_tb.loc[i, 'degtout'] = prop_tb.loc[i, 'tout'] + 273.15 - prop_tb.loc[i, 'tsat'] # [degC]
                    prop_tb.loc[i, 'xe'] = round(prop_tb.loc[i, 'enthout']/prop_tb.loc[i, 'lam'],12)
            else:
                # ONLY degtout exist
                prop_tb.loc[i, 'xe'] = round(prop_tb.loc[i, 'cpf']*prop_tb.loc[i, 'degtout']*1e-3/prop_tb.loc[i, 'lam'],6) #[-]
                prop_tb.loc[i, 'enthout'] = round(prop_tb.loc[i, 'cpf'] * prop_tb.loc[i, 'degtout']*1e-3, 6) # [kJ/kg]
                prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'enthout'] -prop_tb.loc[i, 'hfo'], 12) # [kJ/kg]
                prop_tb.loc[i, 'tout'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtout'] -273.15 # degC
        else:
            # enthout exist
            prop_tb.loc[i, 'xe'] = round(prop_tb.loc[i, 'enthout']/prop_tb.loc[i, 'lam'], 6)
            if prop_tb.loc['enthout'] > 0:
                prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthout'], 12) # [kJ/kg]
                prop_tb.loc[i, 'degtout'] = round((prop_tb.loc[i, 'enthout']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
                prop_tb.loc[i, 'tout'] =  prop_tb.loc[i, 'degtout'] - prop_tb.loc[i, 'tsat'] - 273.15 # degC
            else:
                prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'enthout'] - prop_tb.loc[i, 'hfo'], 12) # [kJ/kg]
                prop_tb.loc[i, 'degtout'] = round((prop_tb.loc[i, 'enthout']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
                prop_tb.loc[i, 'tout'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtout'] -273.15 # degC
    else:
        # Xe exist
        if prop_tb.loc[i, 'xe'] > 0:
            prop_tb.loc[i, 'enthout'] = round(prop_tb.loc[i, 'xe']*prop_tb.loc[i, 'lam'], 12)
            prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'enthout'] -prop_tb.loc[i, 'hfo'], 12) # [kJ/kg]
            prop_tb.loc[i, 'degtout'] = round((prop_tb.loc[i, 'enthout']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
            prop_tb.loc[i, 'tout'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtout'] -273.15 # degC
        else:
            prop_tb.loc[i, 'enthout'] = round(prop_tb.loc[i, 'xe']*prop_tb.loc[i, 'lam'], 12)
            prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthout'], 12) # [kJ/kg]
            prop_tb.loc[i, 'degtout'] = round((prop_tb.loc[i, 'enthout']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
            prop_tb.loc[i, 'tout'] = prop_tb.loc[i, 'degtout'] - prop_tb.loc[i, 'tsat'] -273.15 # degC

    # fill inlet/oulet data
    if pd.isna(prop_tb.loc[i, 'xi']):
            prop_tb.loc[i, 'xi'] = pro.cal_xi(prop_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xe'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 0)
            prop_tb.loc[i, 'enthin'] = round(-prop_tb.loc[i, 'xi']*prop_tb.loc[i, 'lam'], 12)
            prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]
            prop_tb.loc[i, 'degtin'] = round((-prop_tb.loc[i, 'enthin']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
            prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] -273.15 # degC
    else:
        pass

    # Check heat balance
    prop_tb.loc[i,'hval'] = pro.cal_xe(prop_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xi'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 1)
    prop_tb.loc[i,'param_hval'] = round(np.abs(prop_tb.loc[i ,'xe']-prop_tb.loc[i, 'hval']/prop_tb.loc[i, 'hval']), 6)
    if prop_tb.loc[i, 'param_hval'] < 0.05:
        prop_tb.loc[i, 'yn_hval'] = 100
    else:
        prop_tb.loc[i, 'yn_hval'] = 110

# Calculate dimensionless parameters
for i, row in prop_tb.iterrows():

    # Calculate dimensionless number
    prop_tb.loc[i, 'pe'] = round(
        pro.cal_pe(prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf']), 6)
    prop_tb.loc[i, 're'] = round(pro.cal_re(prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'muf']), 6)
    prop_tb.loc[i, 'we'] = round(
        pro.cal_we(prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'vf'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'sigma']), 6)
    prop_tb.loc[i, 'bd'] = round(
        pro.cal_bd(prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'sigma']), 6)
    prop_tb.loc[i, 'pr'] = round(pro.cal_pr(prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf']), 6)
    prop_tb.loc[i, 'prv'] = round(pro.cal_pr(prop_tb.loc[i, 'cpv'], prop_tb.loc[i, 'muv'], prop_tb.loc[i, 'kv']), 6)
    prop_tb.loc[i, 'ca'] = round(
        pro.cal_ca(prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'vf'], prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'rhof']), 6)
    prop_tb.loc[i, 'lc'] = round(np.sqrt(prop_tb.loc[i, 'sigma']/(9.8*(prop_tb.loc[i, 'rhof']-prop_tb.loc[i, 'rhov']))), 6)
    # Calculate old_xosv (Initial condition)
    if m_idx == 1:
        prop_tb.loc[i, 'org_dtsz'], prop_tb.loc[i, 'org_xosv'] = mod.cal_SZ(prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam'])
    else:
        prop_tb.loc[i, 'org_dtsz'], prop_tb.loc[i, 'org_xosv'] = mod.cal_Levy(prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 're'], prop_tb.loc[i, 'pr'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'vf'])
      
# stop the timer
end_time = time.time()
print('END TIME :',str(datetime.now())[10:19])

# calculate the elapsed time
elapsed_time = end_time - start_time

# print the elapsed time
print("Elapsed time to compute the geometrical or dimensionless parameters: {:.2f} mins".format(elapsed_time/60))

# remove local parameters
del start_time, end_time, elapsed_time

START TIME :  19:03:00


C:\Users\PDL-JHW\AppData\Local\Temp\ipykernel_13464\2805161254.py:192: RuntimeWarning: invalid value encountered in scalar divide
  prop_tb.loc[i,'param_hval'] = round(np.abs(prop_tb.loc[i ,'xe']-prop_tb.loc[i, 'hval']/prop_tb.loc[i, 'hval']), 6)


END TIME :  19:08:00
Elapsed time to compute the geometrical or dimensionless parameters: 4.99 mins


In [4]:
# Set the features
columns = ['pre', 'qav', 'gre', 'dhy', 'lh', 'xi', 'xe', 'lpd', 're', 'pr', 'prv', 'qr']

# Initialize a dataframe for saving the boundary values
boundaries_df = pd.DataFrame(columns=['Column', 'Group', 'Min', 'Max'])

# Initialized previous maximum value
prev_max = 0

# K-means clustering and storing group indices and boundaries
for col in columns:
    X = prop_tb[[col]]
    kmeans = KMeans(n_clusters=20, random_state=0)
    labels = kmeans.fit_predict(X)
    prop_tb[f'{col}_Group'] = labels + 1  # Group indices start from 1
    
    # Calculate and store the boundaries of each group
    centers = kmeans.cluster_centers_.flatten()
    centers.sort()
    for i, center in enumerate(centers):
        boundaries = pd.DataFrame({
            'Column': col,
            'Group': i + 1,
            'Min': prev_max + 0.001 if i > 0 else 0,
            'Max': center
        }, index=[i])
        boundaries_df = pd.concat([boundaries_df, boundaries], ignore_index=True)
        prev_max = center
   
# 경계값 데이터프레임 출력
print(prop_tb)

C:\Users\PDL-JHW\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\PDL-JHW\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\PDL-JHW\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn

       idx   tno runno geo flow refri  hs       pre      doi      dio  ...  \
0        1     1     1   C   Up   R12   1 10.686785 0.010947 0.000000  ...   
1        1     2     2   C   Up   R12   1 10.735048 0.010947 0.000000  ...   
2        1     3     3   C   Up   R12   1 10.735048 0.010947 0.000000  ...   
3        1     4     4   C   Up   R12   1 10.824679 0.010947 0.000000  ...   
4        1     5     6   C   Up   R12   1 10.583365 0.010947 0.000000  ...   
...    ...   ...   ...  ..  ...   ...  ..       ...      ...      ...  ...   
20310   60  TC.3    21   A   up   H2O   1  1.184000 0.018000 0.009500  ...   
20311   60  TC.3    22   A   up   H2O   1  1.984000 0.018000 0.009500  ...   
20312   60  TC.3    23   A   up   H2O   1  2.976000 0.018000 0.009500  ...   
20313   60  TC.3    24   A   up   H2O   1  1.190000 0.018000 0.009500  ...   
20314   60  TC.3    25   A   up   H2O   1  1.990000 0.018000 0.009500  ...   

       gre_Group  dhy_Group  lh_Group  xi_Group  xe_Group  lpd_

In [5]:
# create session of database engine
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(json_res["user"][1],json_res["password"][1],json_res["host"][1],json_res["port"][1],"CHF"), echo=False, encoding='utf-8')

# Export results of physical property to MariaDB
prop_tb.to_sql('prop_chf_tb', engine, if_exists='replace')

del engine

In [6]:
# DSM Method (Local condition hypothesis)

dsm_tb = prop_tb.copy()

# Set osv correlation method
m_idx = 1

for i, row in dsm_tb.iterrows():
    # Step 1 calculate Xc
    dsm_tb.loc[i, 'xc'] = pro.cal_xe(dsm_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], dsm_tb.loc[i, 'doi'], dsm_tb.loc[i, 'dio'], dsm_tb.loc[i, 'geo'], dsm_tb.loc[i, 'hs'], dsm_tb.loc[i, 'gre'], dsm_tb.loc[i, 'xi'], dsm_tb.loc[i, 'lh'], dsm_tb.loc[i, 'lam'], ch = 0)

    # Step 1 calculate osv
    if m_idx == 1:
        dsm_tb.loc[i, 'dtsz'], dsm_tb.loc[i, 'xosv'] = mod.cal_SZ(dsm_tb.loc[i, 'qi']+dsm_tb.loc[i, 'qo'], dsm_tb.loc[i, 'rhof'], dsm_tb.loc[i, 'dhy'], dsm_tb.loc[i, 'gre'], dsm_tb.loc[i, 'cpf'], dsm_tb.loc[i, 'kf'], dsm_tb.loc[i, 'pe'], dsm_tb.loc[i, 'lam'])
    else:
        dsm_tb.loc[i, 'dtsz'], dsm_tb.loc[i, 'xosv'] = mod.cal_Levy(dsm_tb.loc[i, 'sigma'], dsm_tb.loc[i, 'dhy'], dsm_tb.loc[i, 'rhof'], dsm_tb.loc[i, 'muf'], dsm_tb.loc[i, 'kf'], dsm_tb.loc[i, 're'], dsm_tb.loc[i, 'pr'], dsm_tb.loc[i, 'cpf'], dsm_tb.loc[i, 'gre'], dsm_tb.loc[i, 'qi']+dsm_tb.loc[i, 'qo'], dsm_tb.loc[i, 'lam'], dsm_tb.loc[i, 'vf'])
    
    # Step 2 calculate Xt
    dsm_tb.loc[i, 'xt'] = round(mod.cal_xt(dsm_tb.loc[i, 'xi'], dsm_tb.loc[i, 'xosv'], dsm_tb.loc[i, 'xc'], dsm_tb.loc[i, 'xc']),6)
 

In [8]:
# HBM Method (Inlet condition hypothesis)
# set iniital calculation method
m_idx = 1
cal_idx = 3
mod_idx = 'Jeong'

# create log table 
log_tb = pd.DataFrame()

# set global parameters
global j
j = 0
tolerance = 0.001

# start the timer
start_time = time.time()
print('START TIME :',str(datetime.now())[10:19] )

for i, row in prop_tb.iterrows():
    if m_idx == 1:
        prop_tb.loc[i, 'mth_xosv'] = 'SZ'
    else:
        prop_tb.loc[i, 'mth_xosv'] = 'Levy'
    
    cnt = 1
    # Preparing alpha and gamma
    prop_tb.loc[i, 'log_id'] = int(i) # Set foreign key to log-table

    # calculate original Xosv from experimental CHF heat flux
    if m_idx == 1:
        prop_tb.loc[i, 'org_dtsz'], prop_tb.loc[i, 'org_xosv'] = mod.cal_SZ(prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam'])
    else:
        prop_tb.loc[i, 'org_dtsz'], prop_tb.loc[i, 'org_xosv'] = mod.cal_Levy(prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 're'], prop_tb.loc[i, 'pr'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'vf'])
    
    while 1:
        # Prepare : log-table
        log_tb.loc[j, 'run_id'] = int(i)
        log_tb.loc[j, 'org_xi'] = prop_tb.loc[i, 'xi']
        log_tb.loc[j, 'org_xe'] = prop_tb.loc[i, 'xe']
        # initialize heat flux
        log_tb.loc[j, 'org_q'] = prop_tb.loc[i, 'qav']
        log_tb.loc[j, 'yn_hval'] = prop_tb.loc[i, 'yn_hval']
        
        # Ref: Original algorithm
            # calculate initial Xt from qc correlation using qexp
            # calculate Xo using qexp = - StGlamXo
            # calculate Xt from rate equation
            # compare oldXt vs. new Xt

        # Step 0: Initialize Xt when the first iteration step
        if cnt == 1:
            if log_tb.loc[j, 'org_xe'] < 0:
                log_tb.loc[j, 'old_xt'] = 0.001
            else:
                log_tb.loc[j, 'old_xt'] = log_tb.loc[j, 'org_xe']
        else:
            pass

        # Step 2: Calculate qc,old geo, hs, doi, dio, dh, p, pcrit, g, q, muf, muv, rhof, rhov, cpf, cpv, v, lam, mass, tcrit, xt_cal=0.5)
        #log_tb.loc[j, 'alpha'], log_tb.loc[j, 'gamma'], log_tb.loc[j, 'zxt'], log_tb.loc[j, 'old_qcal'] = mod.calCHFJeong(prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'pre'], prop_tb.loc[i, 'pcrit'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'muv'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'cpv'], prop_tb.loc[i, 'vf'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'mass'], prop_tb.loc[i, 'tcrit'], log_tb.loc[j, 'old_xt'])

        log_tb.loc[j, 'alpha'], log_tb.loc[j, 'gamma'], log_tb.loc[j, 'zxt'], log_tb.loc[j, 'old_qcal'] = mod.calCHFDeng(prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'rdcp'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], log_tb.loc[j, 'old_xt'])
        
        # Step 3: Calculate old_xosv
        if m_idx == 1:
            log_tb.loc[j, 'old_dtsz'], log_tb.loc[j, 'old_xosv'] = mod.cal_SZ(log_tb.loc[j, 'old_qcal'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam'])
        else:
            log_tb.loc[j, 'old_dtsz'], log_tb.loc[j, 'old_xosv'] = mod.cal_Levy(prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 're'], prop_tb.loc[i, 'pr'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'gre'], log_tb.loc[j, 'old_qcal'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'vf'])

        # Step 4: Calculate old_xe
        log_tb.loc[j, 'old_xe'] = pro.cal_xe(log_tb.loc[j, 'old_qcal'], 0, prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xi'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 1)

        # Step 2: Calculate old_xt (initialization)
        log_tb.loc[j, 'new_xt'] = round(mod.cal_xt(log_tb.loc[j, 'org_xi'], log_tb.loc[j, 'old_xosv'], log_tb.loc[j, 'org_xe'], log_tb.loc[j, 'old_xe']),6)

        # Step 3: Loop control
        cnt += 1 # Increase limitation

        # Step 4: Valudation
        val = abs((log_tb.loc[j, 'new_xt'] - log_tb.loc[j, 'old_xt'])/log_tb.loc[j, 'old_xt'])

        if val <= tolerance:
            # Step 2: Calculate old_xt (initialization)
            prop_tb.loc[i, 'cal_xt'] = round(float(log_tb.loc[j, 'new_xt']),6)
            prop_tb.loc[i, 'cal_xe'] = round(log_tb.loc[j, 'old_xe'], 4)
            prop_tb.loc[i, 'cal_xosv'] = round(log_tb.loc[j, 'old_xosv'], 4)
            prop_tb.loc[i, 'converged'] = 100
            prop_tb.loc[i, 'alpha'], prop_tb.loc[i, 'gamma'], prop_tb.loc[i, 'zxt'], prop_tb.loc[i, 'qcal'] = mod.calCHFDeng(prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'rdcp'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], log_tb.loc[j, 'new_xt'])
            #prop_tb.loc[i, 'alpha'], prop_tb.loc[i, 'gamma'], prop_tb.loc[i, 'zxt'], prop_tb.loc[i, 'qcal'] = mod.calCHFJeong(prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'pre'], prop_tb.loc[i, 'pcrit'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'muv'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'cpv'], prop_tb.loc[i, 'vf'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'mass'], prop_tb.loc[i, 'tcrit'], log_tb.loc[j, 'new_xt'])
            prop_tb.loc[i, 'qval'] = round((prop_tb.loc[i, 'qcal'] - log_tb.loc[j, 'org_q'])/log_tb.loc[j, 'org_q'], 4)
            if abs(prop_tb.loc[i, 'qval']) > 0.20:
                if cnt > 500:
                    print("Data run number : {:d} isn't converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'qav'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
                    del cnt
                    break
                else:
                    log_tb.loc[j+1, 'old_xt'] = 0.99* log_tb.loc[j, 'old_xt'] + 0.01*log_tb.loc[j, 'new_xt']
                    j += 1
                    continue
            else:
                print("Data run number : {:d} is converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'qav'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
                j += 1
                del cnt
                break
        else:
            if cnt > 500:
                prop_tb.loc[i, 'cal_xt'] = round(float(log_tb.loc[j, 'new_xt']),6)
                prop_tb.loc[i, 'cal_xe'] = log_tb.loc[j, 'old_xe']
                prop_tb.loc[i, 'cal_xosv'] = log_tb.loc[j, 'old_xosv']
                prop_tb.loc[i, 'converged'] = 110
                prop_tb.loc[i, 'qval'] = ((prop_tb.loc[i, 'qcal'] - log_tb.loc[j, 'org_q'])/log_tb.loc[j, 'org_q'])
                if abs(prop_tb.loc[i, 'qval']) < 0.20:
                    print("Data run number : {:d} is converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'qav'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
                    prop_tb.loc[i, 'alpha'] = log_tb.loc[j, 'alpha']
                    prop_tb.loc[i, 'gamma'] = log_tb.loc[j, 'gamma']
                    prop_tb.loc[i, 'zxt'] = log_tb.loc[j, 'zxt']
                    prop_tb.loc[i, 'qcal'] = log_tb.loc[j, 'old_qcal']
                    j += 1
                    del cnt
                    break
                else:
                    print("Data run number : {:d} isn't converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'qav'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
                    prop_tb.loc[i, 'alpha'] = log_tb.loc[j, 'alpha']
                    prop_tb.loc[i, 'gamma'] = log_tb.loc[j, 'gamma']
                    prop_tb.loc[i, 'zxt'] = log_tb.loc[j, 'zxt']
                    prop_tb.loc[i, 'qcal'] = log_tb.loc[j, 'old_qcal']
                    j += 1
                    del cnt
                    break
            else:
                # Step 5: Recurrsive state
                log_tb.loc[j+1, 'old_xt'] = 0.9* log_tb.loc[j, 'old_xt'] + 0.1*log_tb.loc[j, 'new_xt']
                j += 1
                continue

# replace nan value to 0
prop_tb.fillna(0, inplace=True)

# stop the timer
end_time = time.time()
print('END TIME :',str(datetime.now())[10:19])

# calculate the elapsed time
elapsed_time = end_time - start_time

# print the elapsed time
print("Elapsed time to compute the physical properties: {:.2f} mins".format(elapsed_time/60))

# delete parameters
del i, j, tolerance, start_time, end_time, elapsed_time

START TIME :  19:15:27
Data run number : 0 isn't converged. Xosv: -0.2476, Xe: 0.7320, Xt: 1.0519, qexp: 0.0737, qcal: 0.1059, and qval: 0.44
Data run number : 1 isn't converged. Xosv: -0.2530, Xe: 0.7300, Xt: 1.0479, qexp: 0.0765, qcal: 0.1086, and qval: 0.42
Data run number : 2 isn't converged. Xosv: -0.2613, Xe: 0.7180, Xt: 1.0426, qexp: 0.0798, qcal: 0.1128, and qval: 0.41
Data run number : 3 isn't converged. Xosv: -0.2744, Xe: 0.6790, Xt: 1.0267, qexp: 0.0856, qcal: 0.1219, and qval: 0.42
Data run number : 4 isn't converged. Xosv: -0.2247, Xe: 0.6470, Xt: 0.9743, qexp: 0.0811, qcal: 0.1232, and qval: 0.52
Data run number : 5 isn't converged. Xosv: -0.2326, Xe: 0.6380, Xt: 0.9706, qexp: 0.0849, qcal: 0.1275, and qval: 0.50
Data run number : 6 isn't converged. Xosv: -0.2466, Xe: 0.6460, Xt: 0.9762, qexp: 0.0896, qcal: 0.1307, and qval: 0.46
Data run number : 7 isn't converged. Xosv: -0.2579, Xe: 0.6020, Xt: 0.9583, qexp: 0.0958, qcal: 0.1417, and qval: 0.48
Data run number : 8 isn't

C:\Users\PDL-JHW\AppData\Local\Temp\ipykernel_13464\965952346.py:80: RuntimeWarning: divide by zero encountered in scalar divide
  val = abs((log_tb.loc[j, 'new_xt'] - log_tb.loc[j, 'old_xt'])/log_tb.loc[j, 'old_xt'])


Data run number : 130 isn't converged. Xosv: -0.3198, Xe: 0.0000, Xt: 0.4123, qexp: 0.2811, qcal: 0.7885, and qval: 1.80
Data run number : 131 isn't converged. Xosv: -0.3742, Xe: 0.0400, Xt: 0.5139, qexp: 0.2426, qcal: 0.6582, and qval: 1.71
Data run number : 132 isn't converged. Xosv: -0.2783, Xe: 0.7320, Xt: 1.0473, qexp: 0.0872, qcal: 0.1225, and qval: 0.41
Data run number : 133 isn't converged. Xosv: -0.2873, Xe: 0.7340, Xt: 1.0357, qexp: 0.0948, qcal: 0.1297, and qval: 0.37
Data run number : 134 isn't converged. Xosv: -0.2951, Xe: 0.7210, Xt: 1.0284, qexp: 0.0990, qcal: 0.1348, and qval: 0.36
Data run number : 135 isn't converged. Xosv: -0.2150, Xe: 0.4730, Xt: 0.8350, qexp: 0.1075, qcal: 0.1871, and qval: 0.74
Data run number : 136 isn't converged. Xosv: -0.2186, Xe: 0.4610, Xt: 0.8276, qexp: 0.1114, qcal: 0.1931, and qval: 0.73
Data run number : 137 isn't converged. Xosv: -0.2300, Xe: 0.4370, Xt: 0.8213, qexp: 0.1178, qcal: 0.2034, and qval: 0.73
Data run number : 138 isn't conv

C:\Users\PDL-JHW\AppData\Local\Temp\ipykernel_13464\965952346.py:80: RuntimeWarning: divide by zero encountered in scalar divide
  val = abs((log_tb.loc[j, 'new_xt'] - log_tb.loc[j, 'old_xt'])/log_tb.loc[j, 'old_xt'])


Data run number : 3529 is converged. Xosv: -0.3080, Xe: 0.0000, Xt: 0.1037, qexp: 4.6673, qcal: 5.5759, and qval: 0.19
Data run number : 3530 is converged. Xosv: -0.3242, Xe: -0.0930, Xt: 0.0679, qexp: 5.7759, qcal: 6.5031, and qval: 0.13
Data run number : 3531 is converged. Xosv: -0.3242, Xe: -0.0880, Xt: 0.0679, qexp: 5.9677, qcal: 6.5031, and qval: 0.09
Data run number : 3532 is converged. Xosv: -0.1715, Xe: -0.0470, Xt: 0.0383, qexp: 5.5432, qcal: 6.4003, and qval: 0.15
Data run number : 3533 is converged. Xosv: -0.1721, Xe: -0.0480, Xt: 0.0384, qexp: 5.5022, qcal: 6.4022, and qval: 0.16
Data run number : 3534 is converged. Xosv: -0.2136, Xe: -0.1350, Xt: 0.0160, qexp: 7.7196, qcal: 8.4811, and qval: 0.10
Data run number : 3535 is converged. Xosv: -0.2140, Xe: -0.1350, Xt: 0.0160, qexp: 7.7196, qcal: 8.4838, and qval: 0.10
Data run number : 3536 is converged. Xosv: -0.3773, Xe: 0.3590, Xt: 0.3682, qexp: 3.1617, qcal: 3.2389, and qval: 0.02
Data run number : 3537 is converged. Xosv:

...

Data run number : 13421 isn't converged. Xosv: -0.4069, Xe: 0.0000, Xt: 0.4714, qexp: 0.3673, qcal: 0.8835, and qval: 1.41
Data run number : 13422 isn't converged. Xosv: -0.3684, Xe: 0.0500, Xt: 0.4763, qexp: 0.3294, qcal: nan, and qval: nan
Data run number : 13423 isn't converged. Xosv: -0.3684, Xe: 0.0500, Xt: 0.4763, qexp: 0.3294, qcal: nan, and qval: nan
Data run number : 13424 isn't converged. Xosv: -0.3231, Xe: 0.1100, Xt: 0.5078, qexp: 0.2720, qcal: 0.7014, and qval: 1.58
Data run number : 13425 isn't converged. Xosv: -0.3231, Xe: 0.1100, Xt: 0.5078, qexp: 0.2720, qcal: 0.7014, and qval: 1.58
Data run number : 13426 isn't converged. Xosv: -0.2924, Xe: 0.1600, Xt: 0.5227, qexp: 0.2385, qcal: 0.6348, and qval: 1.66
Data run number : 13427 isn't converged. Xosv: -0.2924, Xe: 0.1600, Xt: 0.5227, qexp: 0.2385, qcal: 0.6348, and qval: 1.66
Data run number : 13428 isn't converged. Xosv: -0.4988, Xe: 0.1300, Xt: 0.6245, qexp: 0.3345, qcal: 0.6716, and qval: 1.01
Data run number : 13429 

C:\Users\PDL-JHW\AppData\Local\Temp\ipykernel_13464\965952346.py:80: RuntimeWarning: divide by zero encountered in scalar divide
  val = abs((log_tb.loc[j, 'new_xt'] - log_tb.loc[j, 'old_xt'])/log_tb.loc[j, 'old_xt'])


Data run number : 13801 isn't converged. Xosv: -0.3797, Xe: 0.0000, Xt: 0.4376, qexp: 0.4520, qcal: 1.0224, and qval: 1.26


C:\Users\PDL-JHW\AppData\Local\Temp\ipykernel_13464\965952346.py:80: RuntimeWarning: divide by zero encountered in scalar divide
  val = abs((log_tb.loc[j, 'new_xt'] - log_tb.loc[j, 'old_xt'])/log_tb.loc[j, 'old_xt'])


Data run number : 13802 isn't converged. Xosv: -0.3797, Xe: 0.0000, Xt: 0.4376, qexp: 0.4520, qcal: 1.0224, and qval: 1.26
Data run number : 13803 isn't converged. Xosv: -0.3797, Xe: -0.0200, Xt: 0.4376, qexp: 0.4241, qcal: 1.0224, and qval: 1.41
Data run number : 13804 isn't converged. Xosv: -0.3797, Xe: -0.0200, Xt: 0.4376, qexp: 0.4241, qcal: 1.0224, and qval: 1.41
Data run number : 13805 isn't converged. Xosv: -0.3615, Xe: 0.0300, Xt: 0.4453, qexp: 0.4221, qcal: 0.9733, and qval: 1.31
Data run number : 13806 isn't converged. Xosv: -0.3615, Xe: 0.0300, Xt: 0.4453, qexp: 0.4221, qcal: 0.9733, and qval: 1.31
Data run number : 13807 isn't converged. Xosv: -0.3439, Xe: 0.0500, Xt: 0.4529, qexp: 0.3943, qcal: 0.9259, and qval: 1.35
Data run number : 13808 isn't converged. Xosv: -0.3439, Xe: 0.0500, Xt: 0.4529, qexp: 0.3974, qcal: 0.9259, and qval: 1.33
Data run number : 13809 isn't converged. Xosv: -0.3259, Xe: 0.0700, Xt: 0.4610, qexp: 0.3653, qcal: 0.8774, and qval: 1.40
Data run numbe

In [ ]:
# Save result of DSM method
# create engine
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(json_res["user"][1],json_res["password"][1],json_res["host"][1],json_res["port"][1],"CHF"), echo=False, encoding='utf-8')

# export the log- and result-table to MariaDB
dsm_tb.to_sql('res_dsm_tb', engine, if_exists='replace')

# delete parameter
del engine

In [ ]:
# create engine
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(json_res["user"][1],json_res["password"][1],json_res["host"][1],json_res["port"][1],"CHF"), echo=False, encoding='utf-8')
log_tb = log_tb.replace([np.inf, -np.inf], 0)
prop_tb = prop_tb.replace([np.inf, -np.inf], 0)

# export the log- and result-table to MariaDB
log_tb.to_sql('deng_log_chf_tb', engine, if_exists='replace')
prop_tb.to_sql('deng_res_chf_tb', engine, if_exists='replace')

# delete parameter
del engine